In [5]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

In [6]:
# Caminho do arquivo Excel
pathfile = './data/20240801_Planilhas_com_respostas_mdc_OCULTO.xlsx'
# Carrega a aba específica do arquivo
df_pessoas = pd.read_excel(pathfile, sheet_name='20240801_Planilhas_com_resposta')

# Mapeamento do campo Tipo de Desligamento com o Y
tipo_desligamento_dict = pd.read_excel(pathfile, sheet_name='Tipo Desligamento > Y')

# Remove espaços extras e valores nulos de cada coluna
df_categoria = pd.DataFrame(df_pessoas['categoria_mdc'].dropna().astype(str).str.strip().unique(), columns=['categoria_mdc'])
df_conhecimento = pd.DataFrame(df_pessoas['conhecimento_mdc'].dropna().astype(str).str.strip().unique(), columns=['conhecimento_mdc'])
df_habilidade = pd.DataFrame(df_pessoas['habilidade_mdc'].dropna().astype(str).str.strip().unique(), columns=['habilidade_mdc'])


In [7]:
# Caminho do arquivo Excel
pathfile = './data/Pessoas_alocacoes_projetos_OCULTOS_projeto.xlsx'
# Carrega a aba específica do arquivo
df_projetos = pd.read_excel(pathfile, sheet_name='Pessoas_alocacoes_projetos_OCUL')

In [8]:
# Lista de colunas de competências
competencias_cols = [f'competencia{i}' for i in range(1, 22)]

# Junta todas as competências em uma coluna única (dropando valores nulos e duplicados)
competencias = pd.Series(df_projetos[competencias_cols].values.ravel()).dropna()
competencias = competencias.str.strip().drop_duplicates().reset_index(drop=True)

print("🔍 Total de competências únicas:", len(competencias))
print(competencias.head())

🔍 Total de competências únicas: 34
0                   Arquiteturas
1     Compilação e Empacotamento
2                     Containers
3         Gerenciamento de Teste
4    Gestão de dados para testes
dtype: object


In [9]:
from sentence_transformers import SentenceTransformer
import hdbscan

import warnings
warnings.filterwarnings('ignore')

# ============================
# Embeddings com SentenceTransformer
# ============================
modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings = modelo.encode(competencias, show_progress_bar=True)

# ============================
# Agrupamento com HDBSCAN
# ============================
clusterer = hdbscan.HDBSCAN(min_cluster_size=3, metric='euclidean')
labels = clusterer.fit_predict(embeddings)

# Agrupa os resultados
competencias_clustered = pd.DataFrame({
    'competencia': competencias,
    'grupo': labels
})

# Visualiza os grupos encontrados
num_grupos = len(set(labels)) - (1 if -1 in labels else 0)
print(f"\n🧩 Total de grupos encontrados: {num_grupos}")

for grupo in sorted(competencias_clustered['grupo'].unique()):
    if grupo == -1:
        print("\n🔍 Competências não agrupadas (outliers):")
    else:
        print(f"\n🧩 Grupo {grupo}")
    print(competencias_clustered[competencias_clustered['grupo'] == grupo]['competencia'].values)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]


🧩 Total de grupos encontrados: 0

🔍 Competências não agrupadas (outliers):
['Arquiteturas' 'Compilação e Empacotamento' 'Containers'
 'Gerenciamento de Teste' 'Gestão de dados para testes'
 'Integração contínua' 'Linguagens e APIs'
 'Metodologias e tipos de testes' 'Report de Testes' 'RPA' 'Segurança'
 'Testes de API' 'Testes Manuais' 'Testes não funcionais'
 'ALM, Issue tracking e versionamento' 'Análise Estática de Código'
 'Levantamento de Requisitos e Documentação'
 'Normas, Boas Práticas e Métodos' 'Repositório de binários'
 'Testes automatizados e frame' 'Testes Mobile' 'Testes Unitários'
 'Tipos de requisito' 'Web e Application Server / Serverless'
 'Desenvolvimento Mobile' 'Banco de dados' 'UI / UX' 'Monitoramento e APM'
 'Testes de Interface' 'Infraestrutura como Código' 'Clouds'
 'Gerenciamento de API' 'Ambiente e Sistemas Operacionais' 'Agilidade']


In [10]:
import umap.umap_ as umap
import plotly.express as px

# Redução de dimensionalidade com UMAP
umap_2d = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
embeddings_2d = umap_2d.fit_transform(embeddings)

# DataFrame com resultados
df_plot = pd.DataFrame({
    'competencia': competencias,
    'x': embeddings_2d[:, 0],
    'y': embeddings_2d[:, 1],
    'grupo': labels
})

# Visualização interativa com Plotly
fig = px.scatter(
    df_plot,
    x='x',
    y='y',
    color=df_plot['grupo'].astype(str),
    hover_data={'competencia': True, 'grupo': True},
    title='Visualização de Competências Agrupadas (HDBSCAN + UMAP)',
    labels={'grupo': 'Grupo'}
)

fig.update_layout(height=600)
fig.show()

In [ ]:
from sentence_transformers import SentenceTransformer, util

# === Embeddings com modelo de alta performance ===
modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Gera os embeddings
embeddings_categorias = modelo.encode(df_categoria['categoria_mdc'].tolist(), convert_to_tensor=True, show_progress_bar=True)
embeddings_competencias = modelo.encode(competencias.tolist(), convert_to_tensor=True, show_progress_bar=True)

# === Calcula similaridades e encontra a mais próxima para cada categoria ===
competencias_mais_proximas = []

for idx, emb_categoria in enumerate(embeddings_categorias):
    # Calcula similaridade com todas as competências
    similaridades = util.cos_sim(emb_categoria, embeddings_competencias)[0]
    indice_mais_proximo = similaridades.argmax().item()
    competencia_mais_proxima = competencias[indice_mais_proximo]
    competencias_mais_proximas.append(competencia_mais_proxima)

# DataFrame original de competências (garante que seja coluna única)
df_competencias = pd.DataFrame(competencias, columns=['competencia'])

# === Para cada competência, encontra a categoria_mdc mais próxima ===
categorias = df_categoria['categoria_mdc'].tolist()  # mesma lista usada nos embeddings

categorias_mais_proximas = []

for idx, emb_comp in enumerate(embeddings_competencias):
    similaridades = util.cos_sim(emb_comp, embeddings_categorias)[0]
    indice_mais_proximo = similaridades.argmax().item()
    categoria_mais_proxima = categorias[indice_mais_proximo]
    categorias_mais_proximas.append(categoria_mais_proxima)

# Adiciona ao DataFrame de competências
df_competencias['categoria_mais_proxima'] = categorias_mais_proximas

# Exibe os resultados
print(df_competencias.head(10))

# Adiciona ao dataframe
df_categoria['competencia_mais_proxima'] = competencias_mais_proximas

# Exibe resultado
print(df_categoria.head(10))

# Caminho de saída do arquivo Excel
output_path = './output/mapa_competencias_para_categorias.xlsx'
# Salva o resultado em Excel
df_competencias.to_excel(output_path, index=False, sheet_name='mapa_competencias_categorias')

# Caminho de saída do arquivo Excel
output_path = './output/mapa_categorias_para_competencias.xlsx'
# Salva o resultado em Excel
df_categoria.to_excel(output_path, index=False, sheet_name='mapa_categorias_competencias')


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

                      competencia          categoria_mais_proxima
0                    Arquiteturas                    Arquiteturas
1      Compilação e Empacotamento      Compilação e Empacotamento
2                      Containers                      Containers
3          Gerenciamento de Teste          Gerenciamento de Teste
4     Gestão de dados para testes     Gestão de dados para testes
5             Integração contínua             Integração contínua
6               Linguagens e APIs               Linguagens e APIs
7  Metodologias e tipos de testes  Metodologias e tipos de testes
8                Report de Testes                Report de Testes
9                             RPA                             RPA
                               categoria_mdc  \
0        Self-Service BI, Data Science, Etc.   
1                               Arquiteturas   
2                      Gestão de Aplicativos   
3                          Linguagens e APIs   
4  Levantamento de Requisitos e Do

In [15]:
# Limpa os dados: remove nulos, espaços e garante strings
df_pessoas = df_pessoas[['categoria_mdc', 'habilidade_mdc']].dropna()
df_pessoas['categoria_mdc'] = df_pessoas['categoria_mdc'].astype(str).str.strip()
df_pessoas['habilidade_mdc'] = df_pessoas['habilidade_mdc'].astype(str).str.strip()

# Mapeia habilidade -> lista de categorias (únicas)
mapa_habilidades_categorias = (
    df_pessoas
    .groupby('habilidade_mdc')['categoria_mdc']
    .apply(lambda x: ', '.join(sorted(set(x))))
    .reset_index()
    .rename(columns={'categoria_mdc': 'categorias_associadas'})
)

# Exibe o resultado
print(mapa_habilidades_categorias.head(10))

# Caminho de saída do arquivo Excel
output_path = './output/mapa_habilidades_para_categorias.xlsx'
# Salva o resultado em Excel
mapa_habilidades_categorias .to_excel(output_path, index=False, sheet_name='mapa_habilidades_categorias')


                      habilidade_mdc               categorias_associadas
0                          (ver ALM)                 Integração contínua
1                                A/B      Metodologias e tipos de testes
2                                AKS                          Containers
3                                AWS                              Clouds
4                           Acunetix                           Segurança
5                             Ad-hoc                      Testes Manuais
6                           Adobe XD  Prototipação / Design de Interação
7  Agile / Lean Portfolio Management                           Agilidade
8                             Allure                    Report de Testes
9                 Amazon API Gateway                Gerenciamento de API
